# Keras Multi-layer Perceptron

Nothing crazy.

### Imports (from the UNIX Server)

In [1]:
import os
import numpy as np
import tensorflow as tf
from keras import backend as K


class GenericModel:
    """
    Generic Tensorflow Model Class

    Each subclass of this class needs to define the data structure of its weights
    (which should be respected accross methods) and implement the functions below.

    """

    def get_model(self):
        raise NotImplementedError("Subclasses should implement this!")

    def load_weights(self):
        raise NotImplementedError("Subclasses should implement this!")

    def get_weights(self):
        raise NotImplementedError("Subclasses should implement this!")

    def sum_weights(self, weights1, weights2):
        raise NotImplementedError("Subclasses should implement this!")

    def scale_weights(self, weights, factor):
        raise NotImplementedError("Subclasses should implement this!")

    def inverse_scale_weights(self, weights, factor):
        raise NotImplementedError("Subclasses should implement this!")

class GenericKerasModel(GenericModel):
    def set_weights(self, new_weights):
        self.model.set_weights(new_weights)

    def get_weights(self):
        return self.model.get_weights()

    def get_initial_weights(self):
        model = self.build_model()
        return model.get_weights()

    def sum_weights(self, weights1, weights2):
        new_weights = []
        for w1, w2 in zip(weights1, weights2):
            new_weights.append(w1 + w2)
        return new_weights

    def scale_weights(self, weights, factor):
        new_weights = []
        for w in weights:
            new_weights.append(w * factor)
        return new_weights

    def inverse_scale_weights(self, weights, factor):
        new_weights = []
        for w in weights:
            new_weights.append(w / factor)
        return new_weights


Using TensorFlow backend.


In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers


class KerasPerceptron(GenericKerasModel):
    def __init__(self, is_training=False):
        self.n_input = 784
        self.n_hidden1 = 200
        self.n_hidden2 = 200
        self.n_classes = 10
        self.is_training = is_training
        self.model = self.build_model()
        if is_training:
            self.compile_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(self.n_hidden1, input_shape=(self.n_input,), activation='relu', kernel_initializer=keras.initializers.glorot_uniform()))
        model.add(Dense(self.n_hidden2, activation='relu', kernel_initializer=keras.initializers.glorot_uniform()))
        model.add(Dense(self.n_classes, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform()))
        # model.summary()
        return model

    def compile_model(self):
        sgd = optimizers.SGD(lr=0.001)
        self.model.compile(
            optimizer=sgd,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

In [3]:
"""
Custom saving/loading for Keras.
Based on https://github.com/keras-team/keras/blob/master/keras/engine/saving.py.
"""

import json

from keras import optimizers

from keras.models import model_from_json


def model_from_serialized(serialized_model):
    uncompiled_model = model_from_json(serialized_model['architecture'])
    return _load_optimizer(uncompiled_model, serialized_model['optimizer'])

def get_optimizer(model):
    def get_json_type(obj):
        """Serialize any object to a JSON-serializable structure.
        # Arguments
            obj: the object to serialize
        # Returns
            JSON-serializable structure representing `obj`.
        # Raises
            TypeError: if `obj` cannot be serialized.
        """
        # if obj is a serializable Keras class instance
        # e.g. optimizer, layer
        if hasattr(obj, 'get_config'):
            return {'class_name': obj.__class__.__name__,
                    'config': obj.get_config()}

        # if obj is any numpy type
        if type(obj).__module__ == np.__name__:
            if isinstance(obj, np.ndarray):
                return {'type': type(obj),
                        'value': obj.tolist()}
            else:
                return obj.item()

        # misc functions (e.g. loss function)
        if callable(obj):
            return obj.__name__

        # if obj is a python 'type'
        if type(obj).__name__ == type.__name__:
            return obj.__name__

        raise TypeError('Not JSON Serializable:', obj)

    if model.optimizer:
        metadata = {}
        if isinstance(model.optimizer, optimizers.TFOptimizer):
            warnings.warn(
                'TensorFlow optimizers do not '
                'make it possible to access '
                'optimizer attributes or optimizer state '
                'after instantiation. '
                'As a result, we cannot save the optimizer '
                'as part of the model save file.'
                'You will have to compile your model again '
                'after loading it. '
                'Prefer using a Keras optimizer instead '
                '(see keras.io/optimizers).')
        else:
            metadata['training_config'] = json.dumps({
                'optimizer_config': {
                    'class_name': model.optimizer.__class__.__name__,
                    'config': model.optimizer.get_config()
                },
                'loss': model.loss,
                'metrics': model.metrics,
                'sample_weight_mode': model.sample_weight_mode,
                'loss_weights': model.loss_weights,
            }, default=get_json_type)

    return metadata

def _load_optimizer(uncompiled_model, optimizer_metadata):
    custom_objects = {}

    def convert_custom_objects(obj):
        """Handles custom object lookup.
        # Arguments
            obj: object, dict, or list.
        # Returns
            The same structure, where occurrences
                of a custom object name have been replaced
                with the custom object.
        """
        if isinstance(obj, list):
            deserialized = []
            for value in obj:
                deserialized.append(convert_custom_objects(value))
            return deserialized
        if isinstance(obj, dict):
            deserialized = {}
            for key, value in obj.items():
                deserialized[key] = convert_custom_objects(value)
            return deserialized
        if obj in custom_objects:
            return custom_objects[obj]
        return obj

    # instantiate optimizer
    training_config = optimizer_metadata.get('training_config')
    if training_config is None:
        warnings.warn('No training configuration found in save file: '
                      'the model was *not* compiled. '
                      'Compile it manually.')
        return uncompiled_model
    training_config = json.loads(training_config)
    optimizer_config = training_config['optimizer_config']
    optimizer = optimizers.deserialize(optimizer_config,
                                       custom_objects=custom_objects)

    # Recover loss functions and metrics.
    loss = convert_custom_objects(training_config['loss'])
    metrics = convert_custom_objects(training_config['metrics'])
    sample_weight_mode = training_config['sample_weight_mode']
    loss_weights = training_config['loss_weights']

    # Compile model.
    uncompiled_model.compile(optimizer=optimizer,
                             loss=loss,
                             metrics=metrics,
                             loss_weights=loss_weights,
                             sample_weight_mode=sample_weight_mode)

    model = uncompiled_model
    return model


### Setting up model

In [4]:
m = KerasPerceptron(is_training=True)
m.model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2010      
Total params: 199,210
Trainable params: 199,210
Non-trainable params: 0
_________________________________________________________________


### Data loading

In [5]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train.resize((x_train.shape[0], 28 * 28))
x_test.resize((x_test.shape[0], 28 * 28))

### Evaluation (Initial Weights)

#### Preview of weights

In [6]:
m.get_weights(), [a.shape for a in m.get_weights()]

([array([[ 0.06333545,  0.04913887,  0.016052  , ..., -0.00994565,
           0.00749673,  0.0242041 ],
         [-0.07462638,  0.00478362, -0.06386836, ..., -0.00901598,
           0.02959282, -0.04482715],
         [ 0.00602322,  0.0061181 , -0.02080705, ...,  0.06932788,
          -0.03340314, -0.01306256],
         ...,
         [-0.02649657,  0.01674083,  0.00165223, ...,  0.05540611,
          -0.00644033,  0.06428657],
         [ 0.0232241 ,  0.06377034,  0.06853145, ..., -0.02765635,
           0.02512028, -0.02771208],
         [-0.03769262,  0.07497495, -0.03053048, ...,  0.06895617,
           0.03513973, -0.02401147]], dtype=float32),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 

#### Save initialized model

In [7]:
m.model.save('init_mlp_model_with_w.h5')

#### Evaluation

In [21]:
results = m.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 34us/step

Evaluation loss is 14.771605708312988 and accuracy is 0.0696


### Training the model

In [22]:
m.model.fit(x=x_train, y=y_train, batch_size=128, epochs=15)

Epoch 1/15
60000/60000 [==============================] - 2s 29us/step - loss: 9.2731 - acc: 0.4161
Epoch 2/15
60000/60000 [==============================] - 2s 25us/step - loss: 6.9552 - acc: 0.5620
Epoch 3/15
60000/60000 [==============================] - 2s 26us/step - loss: 5.7177 - acc: 0.6346
Epoch 4/15
60000/60000 [==============================] - 2s 25us/step - loss: 4.3244 - acc: 0.7210
Epoch 5/15
60000/60000 [==============================] - 2s 27us/step - loss: 2.8946 - acc: 0.8072
Epoch 6/15
60000/60000 [==============================] - 2s 26us/step - loss: 1.3408 - acc: 0.9016
Epoch 7/15
60000/60000 [==============================] - 2s 26us/step - loss: 0.7052 - acc: 0.9444
Epoch 8/15
60000/60000 [==============================] - 2s 26us/step - loss: 0.5976 - acc: 0.9524
Epoch 9/15
60000/60000 [==============================] - 2s 27us/step - loss: 0.5346 - acc: 0.9577
Epoch 10/15
60000/60000 [==============================] - 2s 26us/step - loss: 0.4617 - acc: 0.9626

### Evaluation (Trained Weights)

In [23]:
results = m.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 26us/step

Evaluation loss is 0.5822803912632375 and accuracy is 0.9523


In [25]:
m.model.get_weights()

[array([[ 0.02598552,  0.06859225,  0.02587155, ..., -0.06236205,
         -0.00336728, -0.06345939],
        [ 0.01683274,  0.01126797,  0.07087256, ..., -0.05513847,
         -0.01288918,  0.04864055],
        [ 0.00084469,  0.00864899, -0.02593485, ...,  0.0373948 ,
         -0.03898612, -0.01434632],
        ...,
        [ 0.05859149, -0.0080386 , -0.06214044, ...,  0.04170146,
         -0.06370826,  0.02507553],
        [ 0.05051669, -0.01537196,  0.04714635, ...,  0.02431303,
          0.05513039, -0.02399365],
        [ 0.06150328, -0.04680308, -0.07061426, ..., -0.01632583,
         -0.04557209,  0.00341774]], dtype=float32),
 array([-7.25412829e-05,  1.26253426e-05, -4.02761034e-05, -2.56994190e-05,
         5.32451895e-06,  3.88020999e-06, -5.90569252e-05, -3.04294081e-05,
        -2.74780068e-05, -7.45112848e-05, -1.10347864e-05, -1.16880750e-04,
        -5.06060715e-05, -4.95811400e-05, -2.62473645e-06, -2.26823468e-05,
        -4.63582292e-05,  1.24027210e-05,  2.18146488e

### Saving the model

Ordered from best to worst.

#### 1. Using the model + optimizer + weights .h5

In [26]:
m.model.save('saved_mlp_model_with_w.h5')

In [27]:
m2 = keras.models.load_model('saved_mlp_model_with_w.h5') 
results = m2.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 36us/step

Evaluation loss is 0.5822803912632375 and accuracy is 0.9523


In [28]:
# This model can continue training!
m2.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m2.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 29us/step - loss: 0.2917 - acc: 0.9769
Epoch 2/5
60000/60000 [==============================] - 2s 27us/step - loss: 0.2637 - acc: 0.9795
Epoch 3/5
60000/60000 [==============================] - 2s 27us/step - loss: 0.2541 - acc: 0.9803
Epoch 4/5
60000/60000 [==============================] - 2s 29us/step - loss: 0.2368 - acc: 0.9823
Epoch 5/5
10000/10000 [==============================] - 0s 30us/step

Evaluation loss is 0.49609962539901464 and accuracy is 0.9604


#### 2. Using a hybrid (JSON for architecture and optimizer + .h5 for weights)

##### Saving

In [29]:
def serialize_model(keras_model):
    if not keras_model._is_compiled:
        raise Exception("Model needs to be compiled first.")
    return {
        'architecture': keras_model.to_json(),
        'optimizer': get_optimizer(keras_model)
    }

In [30]:
serialized_model = serialize_model(m.model)

In [31]:
serialized_model

{'architecture': '{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "Dense", "config": {"name": "dense_4", "trainable": true, "batch_input_shape": [null, 784], "dtype": "float32", "units": 200, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_5", "trainable": true, "units": 200, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "ac

In [32]:
with open('model_w_serializer.json', 'w') as outfile:
    json.dump(serialized_model, outfile)

##### Loading

In [33]:
m5 = model_from_serialized(serialized_model)
assert m5._is_compiled, "Model is not compiled (UNEXPECTED)"

In [34]:
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss on INITIAL MODEL is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 46us/step

Evaluation loss on INITIAL MODEL is 14.130905374145508 and accuracy is 0.1077


In [35]:
m5.load_weights('saved_mlp_weights.h5')
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss on PRETRAINED MODEL is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 25us/step

Evaluation loss on PRETRAINED MODEL is 0.6063993921547932 and accuracy is 0.9518


In [36]:
# This model can continue training like we want it.
m5.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m5.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.4071 - acc: 0.9687
Epoch 2/5
60000/60000 [==============================] - 2s 28us/step - loss: 0.3599 - acc: 0.9724
Epoch 3/5
60000/60000 [==============================] - 2s 25us/step - loss: 0.3308 - acc: 0.9744
Epoch 4/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.2995 - acc: 0.9768
Epoch 5/5
10000/10000 [==============================] - 0s 31us/step

Evaluation loss is 0.5075688947218925 and accuracy is 0.9589


#### 3. Using the (only) weights .h5 

In [37]:
m.model.save_weights('saved_mlp_weights.h5')

In [38]:
m4 = KerasPerceptron(is_training=True)
m4.model.load_weights('saved_mlp_weights.h5')
results = m4.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 1s 66us/step

Evaluation loss is 0.5822803912632375 and accuracy is 0.9523


In [39]:
# This model can continue training, BUT only because we instantiated the KerasPerceptron class.
m4.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m4.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 2s 38us/step - loss: 0.2830 - acc: 0.9778
Epoch 2/5
60000/60000 [==============================] - 2s 36us/step - loss: 0.2676 - acc: 0.9787
Epoch 3/5
60000/60000 [==============================] - 2s 33us/step - loss: 0.2533 - acc: 0.9803
Epoch 4/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.2421 - acc: 0.9810
Epoch 5/5
10000/10000 [==============================] - 0s 30us/step

Evaluation loss is 0.47415640808192366 and accuracy is 0.9577


#### 4. Using the .npy

In [40]:
trained_weights = m.get_weights()
np.save('./perceptron_trained.npy', trained_weights)

In [41]:
# The accuracy is exactly what it was before :)
m3 = KerasPerceptron(is_training=True)
m3.set_weights(np.load('perceptron_trained.npy'))
results = m3.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

10000/10000 [==============================] - 0s 40us/step

Evaluation loss is 0.5822803912632375 and accuracy is 0.9523


In [42]:
# This model can continue training, BUT only because we instantiated the KerasPerceptron class.
m3.model.fit(x=x_train, y=y_train, batch_size=128, epochs=5)
results = m3.model.evaluate(x=x_test, y=y_test)
print("\nEvaluation loss is {0} and accuracy is {1}".format(*results))

Epoch 1/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.2867 - acc: 0.9770
Epoch 2/5
60000/60000 [==============================] - 2s 36us/step - loss: 0.2728 - acc: 0.9788
Epoch 3/5
60000/60000 [==============================] - 2s 30us/step - loss: 0.2552 - acc: 0.9801
Epoch 4/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.2412 - acc: 0.9814
Epoch 5/5
10000/10000 [==============================] - 0s 25us/step

Evaluation loss is 0.4945828144242561 and accuracy is 0.9589


### Conclusion

When using Keras, the best way to encode the model (and optimizer & weights) and send it around for more training is to use the `model.model.save(filepath)` function. 

**Please notice the** `model.model` **because it's important! Without it the weights will change and the model won't be accurate.**

The exact lines of code to do this would be:

```python
# Serialize the model, optimizer, and weights
model = # The Keras model here
filepath = 'models/filename.h5'
model.model.save(filepath)
```

At this point, `serialized_model` will contain the model and optimizer while the file `weights/filename.h5` will contain the weights.

To send this information through an HTTP request we would do the following:

```python
url = 'https://servers.dataagora.com/start-dml'
metadata = {} # Some metadata
with open(weights_filepath, 'rb') as f:
    r = requests.post(url, files={'file': f}, data=metadata)
```

**NOTES**: It may be that the file we're sending is too big for either the client to send or the server to receive. In these cases, we can either stream or cut the file into chunks to send. Here are some resources to accomplish this: [one](https://stackoverflow.com/a/54857411), [two](https://stackoverflow.com/a/35784072), [three](http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file_).

#### UPDATE

The conclusion above is accurate and probably necessary when using a *TensorFlow.js* trainer, but it's not sufficient. *TensorFlow.js* doesn't incorporate the optimizer information when deserializing the model, so the client needs to manually compile the model. (See the `KerasAndTFJS.ipynb` for more details.)